In [2]:
# Import our dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [4]:
# Import and read the loan_data_cleaned.csv.
loan_data_df = pd.read_csv("./Resources/loan_data_cleaned.csv")  
loan_data_df.head()


,Unnamed: 0,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,0,1,2,Graduate,No,96000,299000,12,778,24000,176000,227000,80000,Approved
1,1,2,0,Not Graduate,Yes,41000,122000,8,417,27000,22000,88000,33000,Rejected
2,2,3,3,Graduate,No,91000,297000,20,506,71000,45000,333000,128000,Rejected
3,3,4,3,Graduate,No,82000,307000,8,467,182000,33000,233000,79000,Rejected
4,4,5,5,Not Graduate,Yes,98000,242000,20,382,124000,82000,294000,50000,Rejected


In [16]:
# Drop loan_id
loan_data_df = loan_data_df.drop("loan_id", axis=1)

# Handle missing values (if any). 
loan_data_df.fillna(loan_data_df.mean(), inplace=True)


C:\Users\NCout\AppData\Local\Temp\ipykernel_11432\3317597038.py:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  loan_data_df.fillna(loan_data_df.mean(), inplace=True)


In [17]:
loan_data_df.head()

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


In [7]:
unique_counts = loan_data_df.nunique()
print(unique_counts)

Unnamed: 0                   4269
loan_id                      4269
 no_of_dependents               6
 education                      2
 self_employed                  2
 income_annum                  98
 loan_amount                  378
 loan_term                     10
 cibil_score                  601
 residential_assets_value     278
 commercial_assets_value      188
 luxury_assets_value          379
 bank_asset_value             146
 loan_status                    2
dtype: int64


In [8]:
# Check the distribution
print(loan_data_df[' cibil_score'].describe())


count    4269.000000
mean        6.353741
std         0.306425
min         5.707110
25%         6.118097
50%         6.398595
75%         6.618739
max         6.803505
Name:  cibil_score, dtype: float64


In [9]:
# Outlier Removal: For numerical column cibil_score
loan_data_df[' cibil_score'] = np.log(loan_data_df[' cibil_score'] + 1)

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(loan_data_df)
print(loan_data_df)

      Unnamed: 0  loan_id   no_of_dependents      education  self_employed  \
0              0        1                  2       Graduate             No   
1              1        2                  0   Not Graduate            Yes   
2              2        3                  3       Graduate             No   
3              3        4                  3       Graduate             No   
4              4        5                  5   Not Graduate            Yes   
...          ...      ...                ...            ...            ...   
4264        4264     4265                  5       Graduate            Yes   
4265        4265     4266                  0   Not Graduate            Yes   
4266        4266     4267                  2   Not Graduate             No   
4267        4267     4268                  1   Not Graduate             No   
4268        4268     4269                  1       Graduate             No   

       income_annum   loan_amount   loan_term   cibil_score  \


In [11]:
label_encoders = {}
for column in [' education', ' self_employed', ' loan_status']:
    le = LabelEncoder()
    loan_data_df[column] = le.fit_transform(loan_data_df[column])
    label_encoders[column] = le

In [13]:
# Splitting the data
X = loan_data_df.drop(" loan_status", axis=1)
y = loan_data_df[" loan_status"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardize the numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer with L2 regularization
nn.add(tf.keras.layers.Dense(units=number_input_features*2, activation="relu", kernel_regularizer=l2(0.01), input_dim=number_input_features))
nn.add(tf.keras.layers.Dropout(0.2))  # 20% dropout

# Second hidden layer with L2 regularization
nn.add(tf.keras.layers.Dense(units=number_input_features, activation="relu", kernel_regularizer=l2(0.01)))
nn.add(tf.keras.layers.Dropout(0.2))  # 20% dropout

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 26)                364       
                                                                 
 dropout_2 (Dropout)         (None, 26)                0         
                                                                 
 dense_4 (Dense)             (None, 13)                351       
                                                                 
 dropout_3 (Dropout)         (None, 13)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 14        
                                                                 
Total params: 729 (2.85 KB)
Trainable params: 729 (2.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Create callback for early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define the optimizer
optimizer = Adam(learning_rate=0.001)

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [17]:
# Train the model
history = nn.fit(
    X_train_scaled,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_scaled, y_test),
    verbose=1,
    callbacks=[early_stopping]
)


Epoch 1/100
101/101 [==============================] - 1s 3ms/step - loss: 0.9351 - accuracy: 0.6364 - val_loss: 0.7910 - val_accuracy: 0.7846
Epoch 2/100
101/101 [==============================] - 0s 1ms/step - loss: 0.7072 - accuracy: 0.7976 - val_loss: 0.5700 - val_accuracy: 0.8820
Epoch 3/100
101/101 [==============================] - 0s 1ms/step - loss: 0.5438 - accuracy: 0.8660 - val_loss: 0.4548 - val_accuracy: 0.9139
Epoch 4/100
101/101 [==============================] - 0s 1ms/step - loss: 0.4462 - accuracy: 0.8991 - val_loss: 0.3931 - val_accuracy: 0.9176
Epoch 5/100
101/101 [==============================] - 0s 1ms/step - loss: 0.3931 - accuracy: 0.9094 - val_loss: 0.3558 - val_accuracy: 0.9213
Epoch 6/100
101/101 [==============================] - 0s 1ms/step - loss: 0.3590 - accuracy: 0.9219 - val_loss: 0.3285 - val_accuracy: 0.9279
Epoch 7/100
101/101 [==============================] - 0s 1ms/step - loss: 0.3428 - accuracy: 0.9206 - val_loss: 0.3084 - val_accuracy: 0.9260

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

34/34 - 0s - loss: 0.1386 - accuracy: 0.9654 - 40ms/epoch - 1ms/step
Loss: 0.1385868936777115, Accuracy: 0.9653558135032654


In [19]:
# Save the trained model
#nn.save("loan_approval_model.h5")
#print("Model saved to loan_approval_model.h5")

In [20]:
# Define an optimized deep neural network model
number_input_features = len(X_train_scaled[0])

optimized_nn = tf.keras.models.Sequential()

# First hidden layer with L2 regularization and swish activation
optimized_nn.add(tf.keras.layers.Dense(units=number_input_features*3, activation="swish", kernel_regularizer=l2(0.01), input_dim=number_input_features))
optimized_nn.add(tf.keras.layers.Dropout(0.3))  # 30% dropout

# Second hidden layer with L2 regularization and swish activation
optimized_nn.add(tf.keras.layers.Dense(units=number_input_features*2, activation="swish", kernel_regularizer=l2(0.01)))
optimized_nn.add(tf.keras.layers.Dropout(0.3))  # 30% dropout

# Third hidden layer
optimized_nn.add(tf.keras.layers.Dense(units=number_input_features, activation="relu"))

# Output layer
optimized_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#display
optimized_nn.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 39)                546       
                                                                 
 dropout_4 (Dropout)         (None, 39)                0         
                                                                 
 dense_7 (Dense)             (None, 26)                1040      
                                                                 
 dropout_5 (Dropout)         (None, 26)                0         
                                                                 
 dense_8 (Dense)             (None, 13)                351       
                                                                 
 dense_9 (Dense)             (None, 1)                 14        
                                                                 
Total params: 1951 (7.62 KB)
Trainable params: 1951 (7

In [21]:
# Create callback for early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Define the optimizer with a learning rate decay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = Adam(learning_rate=lr_schedule)

# Compile the model
optimized_nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [22]:
# Train the model
history = optimized_nn.fit(
    X_train_scaled,
    y_train,
    epochs=150,
    batch_size=64,
    validation_data=(X_test_scaled, y_test),
    verbose=1,
    callbacks=[early_stopping]
)


Epoch 1/150
51/51 [==============================] - 1s 4ms/step - loss: 0.5463 - accuracy: 0.8472 - val_loss: 0.3064 - val_accuracy: 0.9045
Epoch 2/150
51/51 [==============================] - 0s 2ms/step - loss: 0.3051 - accuracy: 0.9153 - val_loss: 0.2902 - val_accuracy: 0.9148
Epoch 3/150
51/51 [==============================] - 0s 2ms/step - loss: 0.2762 - accuracy: 0.9210 - val_loss: 0.2598 - val_accuracy: 0.9242
Epoch 4/150
51/51 [==============================] - 0s 3ms/step - loss: 0.2701 - accuracy: 0.9206 - val_loss: 0.2520 - val_accuracy: 0.9279
Epoch 5/150
51/51 [==============================] - 0s 2ms/step - loss: 0.2795 - accuracy: 0.9132 - val_loss: 0.2511 - val_accuracy: 0.9270
Epoch 6/150
51/51 [==============================] - 0s 2ms/step - loss: 0.2738 - accuracy: 0.9203 - val_loss: 0.2605 - val_accuracy: 0.9270
Epoch 7/150
51/51 [==============================] - 0s 2ms/step - loss: 0.2740 - accuracy: 0.9185 - val_loss: 0.2511 - val_accuracy: 0.9110
Epoch 8/150
5

In [23]:

# Evaluate the model using the test data
model_loss, model_accuracy = optimized_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

34/34 - 0s - loss: 0.1931 - accuracy: 0.9457 - 39ms/epoch - 1ms/step
Loss: 0.19305168092250824, Accuracy: 0.9456928968429565


In [25]:
# Define the model
number_input_features = len(X_train_scaled[0])

advanced_nn = tf.keras.models.Sequential()

# Input and first hidden layer with LeakyReLU
advanced_nn.add(Dense(units=number_input_features*3, input_dim=number_input_features))
advanced_nn.add(LeakyReLU(alpha=0.01))

# Second hidden layer with LeakyReLU
advanced_nn.add(Dense(units=number_input_features*2))
advanced_nn.add(LeakyReLU(alpha=0.01))

# Third hidden layer with tanh
advanced_nn.add(Dense(units=number_input_features, activation='tanh'))

# Fourth hidden layer with tanh
advanced_nn.add(Dense(units=number_input_features//2, activation='tanh'))

# Output layer
advanced_nn.add(Dense(units=1, activation="sigmoid"))

#Display
advanced_nn.summary()



Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 39)                546       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 39)                0         
                                                                 
 dense_16 (Dense)            (None, 26)                1040      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 26)                0         
                                                                 
 dense_17 (Dense)            (None, 13)                351       
                                                                 
 dense_18 (Dense)            (None, 6)                 84        
                                                                 
 dense_19 (Dense)            (None, 1)                

In [26]:
# Compile the model with an adaptive learning rate
optimizer = Adam(learning_rate=0.001)
advanced_nn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [27]:
# Train the model
num_epochs = 150
history = advanced_nn.fit(
    X_train_scaled,
    y_train,
    epochs=num_epochs,
    batch_size=32,
    validation_data=(X_test_scaled, y_test),
    verbose=1
)

Epoch 1/150
101/101 [==============================] - 1s 2ms/step - loss: 0.4691 - accuracy: 0.7982 - val_loss: 0.3051 - val_accuracy: 0.8998
Epoch 2/150
101/101 [==============================] - 0s 1ms/step - loss: 0.2585 - accuracy: 0.9185 - val_loss: 0.2367 - val_accuracy: 0.9204
Epoch 3/150
101/101 [==============================] - 0s 1ms/step - loss: 0.2134 - accuracy: 0.9322 - val_loss: 0.2152 - val_accuracy: 0.9288
Epoch 4/150
101/101 [==============================] - 0s 1ms/step - loss: 0.1947 - accuracy: 0.9356 - val_loss: 0.1985 - val_accuracy: 0.9316
Epoch 5/150
101/101 [==============================] - 0s 1ms/step - loss: 0.1825 - accuracy: 0.9381 - val_loss: 0.1884 - val_accuracy: 0.9316
Epoch 6/150
101/101 [==============================] - 0s 1ms/step - loss: 0.1694 - accuracy: 0.9416 - val_loss: 0.1762 - val_accuracy: 0.9326
Epoch 7/150
101/101 [==============================] - 0s 1ms/step - loss: 0.1564 - accuracy: 0.9444 - val_loss: 0.1582 - val_accuracy: 0.9354

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = advanced_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


34/34 - 0s - loss: 0.1454 - accuracy: 0.9579 - 39ms/epoch - 1ms/step
Loss: 0.14542284607887268, Accuracy: 0.9578651785850525


In [39]:
import pandas as pd

# Capture the model summaries as strings
from io import StringIO
import sys

def get_model_summary(model):
    stream = StringIO()
    sys.stdout = stream
    model.summary()
    sys.stdout = sys.__stdout__
    return stream.getvalue()

# Get model summaries
nn_summary = get_model_summary(nn)
optimized_nn_summary = get_model_summary(optimized_nn)
advanced_nn_summary = get_model_summary(advanced_nn)

# Data for each model
data = {
    'Model': ['nn', 'optimized_nn', 'advanced_nn'],
    'Summary': [nn_summary, optimized_nn_summary, advanced_nn_summary],
    'Loss': [0.1478002965450287, 1.0808980464935303, 0.11039558053016663],
    'Accuracy': [0.9700374603271484, 0.6385768055915833, 0.9691011309623718]
}

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('Deep_model_comparison.csv', index=False)


In [43]:
import plotly.graph_objects as go

# Data for each model
models = ['nn', 'optimized_nn', 'advanced_nn']
accuracies = [97.00374603271484, 63.85768055915833, 96.91011309623718]  
losses = [14.78002965450287, 108.08980464935303, 11.039558053016663]  

# Create traces for accuracies and losses
trace1 = go.Bar(
    x=models,
    y=accuracies,
    name='Accuracy',
    marker=dict(color=['blue', 'green', 'red']),
    hoverinfo='y',
    text=accuracies,
    textposition='auto',
    hovertemplate=
    '<b>Model</b>: %{x}<br><b>Accuracy</b>: %{y:.2f}%<extra></extra>'  # Display value as percentage
)

trace2 = go.Bar(
    x=models,
    y=losses,
    name='Loss',
    marker=dict(color=['lightblue', 'lightgreen', 'lightcoral']),
    hoverinfo='y',
    text=losses,
    textposition='auto',
    hovertemplate=
    '<b>Model</b>: %{x}<br><b>Loss</b>: %{y:.2f}%<extra></extra>'  # Display value as percentage
)

# Create the layout and add traces
layout = go.Layout(
    title='Comparison of Accuracy and Loss for Models',
    barmode='group'
)

fig = go.Figure(data=[trace1, trace2], layout=layout)

# Show plot
fig.show()

